# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.cluster import KMeans
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

# Load and view dataset

In [2]:
df = pd.read_csv('Final_invoice_copy.csv')
pd.set_option('display.max_columns', 100)
df.head(10)

Cust Type Customer No.     District Invoice Date  Invoice No Invoice Time  \
0    Retail        67849  Maharashtra   02-01-2015  7005200002     11:30:36   
1    Retail        84419  Maharashtra   03-01-2015  7005200003     10:07:32   
2    Retail        81055  Maharashtra   03-01-2015  7005200004     11:12:57   
3    Retail        84419  Maharashtra   03-01-2015  7005200005     11:40:44   
4    Retail        18980  Maharashtra   03-01-2015  7005200006     12:07:17   
5    Retail        84457  Maharashtra   03-01-2015  7005200007     12:10:08   
6    Retail        12523  Maharashtra   03-01-2015  7005200008     12:12:31   
7    Retail        55220  Maharashtra   03-01-2015  7005200009     12:18:54   
8    Retail        84511  Maharashtra   03-01-2015  7005200010     13:07:27   
9    Retail        84457  Maharashtra   03-01-2015  7005200011     15:54:21   

   Job Card No JobCard Date JobCard Time  KMs Reading  Labour Total  \
0       168303   13-12-2014     14:29:43        49317       1203.14   
1       173997   02-01-2015     14:12:18        78584        804.26   
2       173889   02-01-2015     11:40:44        33985        180.19   
3       174158   03-01-2015     10:12:32        78584          0.00   
4       173860   02-01-2015     10:45:42        50057       1202.97   
5       174053   02-01-2015     15:15:10        13122          0.00   
6       173953   02-01-2015     12:15:55        26964          0.00   
7       174254   03-01-2015     12:02:32        12943          0.00   
8       174162   03-01-2015     10:21:26        18563          0.00   
9       174203   03-01-2015     11:11:00         5443        698.92   

             Make  Misc Total        Model  OSL Total         Order Type  \
0  GENERAL MOTORS        0.00        SPARK     500.06       Paid Service   
1     TATA MOTORS      197.03       INDICA       0.00  SMC Value Package   
2   MARUTI SUZUKI        0.00          ZEN       0.00    Running Repairs   
3     TATA MOTORS        0.00       INDICA       0.00     SMC Redemption   
4         HYUNDAI      100.00  SANTRO XING     499.99       Paid Service   
5          TOYOTA        0.00       INNOVA     851.70    Running Repairs   
6           HONDA        0.00         CITY     749.42    Running Repairs   
7   MARUTI SUZUKI        0.00  SWIFT DZIRE       0.00    Running Repairs   
8     TATA MOTORS        0.00       INDIGO       0.00    Running Repairs   
9   MARUTI SUZUKI       49.99         ALTO     499.95       Paid Service   

   Parts Total  Recovrbl Exp Total Amt Wtd Tax.   City  
0      2348.75           0.0              4,052  thane  
1         0.00           0.0              1,001  thane  
2        52.95           0.0                233  thane  
3         0.00           0.0               -     thane  
4       944.16           0.0              2,747  thane  
5        14.22           0.0                866  thane  
6         0.00           0.0                749  thane  
7         0.00           0.0               -     thane  
8         0.00           0.0               -     thane  
9      2663.75           0.0              3,913  thane

## Check for missing/null values

In [3]:
df.isnull().sum()

Cust Type                0
Customer No.             0
District                 0
Invoice Date             0
Invoice No               0
Invoice Time             0
Job Card No              0
JobCard Date             0
JobCard Time             0
KMs Reading              0
Labour Total             0
Make                     0
Misc Total               0
Model                 1525
OSL Total                0
Order Type               0
Parts Total              0
Recovrbl Exp             0
Total Amt Wtd Tax.       0
City                     0
dtype: int64

In [4]:
#Join Invoice Date and time
#Join jobcard Date and time
format_str = '%Y-%m-%d %H:%M:%S'

df['invoice_date_time'] = df['Invoice Date'] + ' ' + df['Invoice Time']
df['invoice_date_time'] = pd.to_datetime(df['invoice_date_time'])
df['invoice_date_time'] = df['invoice_date_time'].astype(str).apply(lambda x: datetime.strptime(x, format_str))
df['jobcard_date_time'] = df['JobCard Date'] + ' ' + df['JobCard Time']
df['jobcard_date_time'] = pd.to_datetime(df['jobcard_date_time'])
df['jobcard_date_time'] = df['jobcard_date_time'].astype(str).apply(lambda x: datetime.strptime(x, format_str))
df.head()

Cust Type Customer No.     District Invoice Date  Invoice No Invoice Time  \
0    Retail        67849  Maharashtra   02-01-2015  7005200002     11:30:36   
1    Retail        84419  Maharashtra   03-01-2015  7005200003     10:07:32   
2    Retail        81055  Maharashtra   03-01-2015  7005200004     11:12:57   
3    Retail        84419  Maharashtra   03-01-2015  7005200005     11:40:44   
4    Retail        18980  Maharashtra   03-01-2015  7005200006     12:07:17   

   Job Card No JobCard Date JobCard Time  KMs Reading  Labour Total  \
0       168303   13-12-2014     14:29:43        49317       1203.14   
1       173997   02-01-2015     14:12:18        78584        804.26   
2       173889   02-01-2015     11:40:44        33985        180.19   
3       174158   03-01-2015     10:12:32        78584          0.00   
4       173860   02-01-2015     10:45:42        50057       1202.97   

             Make  Misc Total        Model  OSL Total         Order Type  \
0  GENERAL MOTORS        0.00        SPARK     500.06       Paid Service   
1     TATA MOTORS      197.03       INDICA       0.00  SMC Value Package   
2   MARUTI SUZUKI        0.00          ZEN       0.00    Running Repairs   
3     TATA MOTORS        0.00       INDICA       0.00     SMC Redemption   
4         HYUNDAI      100.00  SANTRO XING     499.99       Paid Service   

   Parts Total  Recovrbl Exp Total Amt Wtd Tax.   City   invoice_date_time  \
0      2348.75           0.0              4,052  thane 2015-02-01 11:30:36   
1         0.00           0.0              1,001  thane 2015-03-01 10:07:32   
2        52.95           0.0                233  thane 2015-03-01 11:12:57   
3         0.00           0.0               -     thane 2015-03-01 11:40:44   
4       944.16           0.0              2,747  thane 2015-03-01 12:07:17   

    jobcard_date_time  
0 2014-12-13 14:29:43  
1 2015-02-01 14:12:18  
2 2015-02-01 11:40:44  
3 2015-03-01 10:12:32  
4 2015-02-01 10:45:42

In [5]:
#Make column for total service time taken
df['service_time'] = (df['invoice_date_time'] - df['jobcard_date_time'])
df['service_time_hours'] = df['service_time'].apply(lambda x: round(x/np.timedelta64(1, 'h'),3))

#Convert service_time to days
df['service_time_days'] = df['service_time'].apply(lambda x: round(x/np.timedelta64(1, 'D'),0))

df.head()

Cust Type Customer No.     District Invoice Date  Invoice No Invoice Time  \
0    Retail        67849  Maharashtra   02-01-2015  7005200002     11:30:36   
1    Retail        84419  Maharashtra   03-01-2015  7005200003     10:07:32   
2    Retail        81055  Maharashtra   03-01-2015  7005200004     11:12:57   
3    Retail        84419  Maharashtra   03-01-2015  7005200005     11:40:44   
4    Retail        18980  Maharashtra   03-01-2015  7005200006     12:07:17   

   Job Card No JobCard Date JobCard Time  KMs Reading  Labour Total  \
0       168303   13-12-2014     14:29:43        49317       1203.14   
1       173997   02-01-2015     14:12:18        78584        804.26   
2       173889   02-01-2015     11:40:44        33985        180.19   
3       174158   03-01-2015     10:12:32        78584          0.00   
4       173860   02-01-2015     10:45:42        50057       1202.97   

             Make  Misc Total        Model  OSL Total         Order Type  \
0  GENERAL MOTORS        0.00        SPARK     500.06       Paid Service   
1     TATA MOTORS      197.03       INDICA       0.00  SMC Value Package   
2   MARUTI SUZUKI        0.00          ZEN       0.00    Running Repairs   
3     TATA MOTORS        0.00       INDICA       0.00     SMC Redemption   
4         HYUNDAI      100.00  SANTRO XING     499.99       Paid Service   

   Parts Total  Recovrbl Exp Total Amt Wtd Tax.   City   invoice_date_time  \
0      2348.75           0.0              4,052  thane 2015-02-01 11:30:36   
1         0.00           0.0              1,001  thane 2015-03-01 10:07:32   
2        52.95           0.0                233  thane 2015-03-01 11:12:57   
3         0.00           0.0               -     thane 2015-03-01 11:40:44   
4       944.16           0.0              2,747  thane 2015-03-01 12:07:17   

    jobcard_date_time     service_time  service_time_hours  service_time_days  
0 2014-12-13 14:29:43 49 days 21:00:53            1197.015               50.0  
1 2015-02-01 14:12:18 27 days 19:55:14             667.921               28.0  
2 2015-02-01 11:40:44 27 days 23:32:13             671.537               28.0  
3 2015-03-01 10:12:32  0 days 01:28:12               1.470                0.0  
4 2015-02-01 10:45:42 28 days 01:21:35             673.360               28.0

In [6]:
#Make new total column and drop existing total
df = df.drop(['Total Amt Wtd Tax.'], 1)
df['Total'] = df['Labour Total'] + df['Misc Total'] + df['OSL Total'] + df['Parts Total']

In [7]:
#Define function to make pivot_table and bar graph
def make_pivot_bar(dataframe, index, values, aggfunc, graph_type, color):
    global pivot_table
    pivot_table = pd.pivot_table(data = dataframe, index = index, values = values,
                                 aggfunc = aggfunc).reset_index().sort_values(by = values, ascending = False)
    fig = px.bar(pivot_table, x = pivot_table[index], y = pivot_table[values], color = color)
    title = "{} Amount spent by {}.".format(graph_type,index)
    fig.update_layout(title = title, width = 1000, height = 900)
    fig.show()

In [8]:
#Bar graph for total amount spent by make
make_pivot_bar(df, 'Make', 'Total', 'sum', 'Total', 'Total')

In [9]:
#Bar graph for total amount spent by district
make_pivot_bar(df, 'District', 'Total', 'sum', 'Total', 'Total')

In [10]:
#Bar graph for average amount spent by district
make_pivot_bar(df, 'District', 'Total', np.mean, 'Average', 'Total')

In [11]:
#Bar graph for average amount spent by make
make_pivot_bar(df, 'Make', 'Total', np.mean, 'Average', 'Total')

In [22]:
#Define function to make pivot_table and bar graph for top 'n' values
def make_pivot_bar_top(dataframe, index, values, aggfunc, n, color):
    global pivot_table
    pivot_table = pd.pivot_table(data = dataframe, index = index, values = values,
                                 aggfunc = aggfunc).reset_index().sort_values(by = values, ascending = False)
    pivot_table = pivot_table.head(n)
    fig = px.bar(pivot_table, x = pivot_table[index], y = pivot_table[values], color = color)
    title = "Highest amount spent by {}.".format(index)
    fig.update_layout(title = title, width = 1000, height = 900)
    fig.show()

In [23]:
#Most profitable model
make_pivot_bar_top(df, 'Model', 'Total', np.mean, 20, 'Total')

In [24]:
#Most profitable Plant
make_pivot_bar_top(df, 'City', 'Total', np.mean, 20, 'Total')

In [25]:
#Most profitable order type
make_pivot_bar_top(df, 'Order Type', 'Total', np.mean, 20, 'Total')

In [26]:
#Most profitable customer type
make_pivot_bar_top(df, 'Cust Type', 'Total', np.mean, 20, 'Total')

In [28]:
#Make column to determine age of the car
average_yearly_km = 10000
df['car_age'] = df['KMs Reading']//average_yearly_km

In [29]:
#Make bins for KMs travelled
df['km_bins'] = pd.cut(x=df['KMs Reading'], bins=[0, 10000, 20000, 50000, 100000, 200000, 500000, np.inf],
                      labels = ['0 - 10000', '10000 - 20000', '20000 - 50000', '50000 - 100000', '100000 - 200000',
                                '200000 - 500000', '>500000'])

In [57]:
#Bar graph for total amount spent by make
make_pivot_bar(df, 'km_bins', 'Total', np.mean, 'Average', 'Total')

In [33]:
#Define function to make pivot_table and bar graph for top 'n' values ascending
def make_pivot_bar_top_garage(dataframe, index, values, aggfunc, n, color):
    global pivot_table
    pivot_table = pd.pivot_table(data = dataframe, index = index, values = values,
                                 aggfunc = aggfunc).reset_index().sort_values(by = values, ascending = True)
    pivot_table = pivot_table[pivot_table[values] > 0]
    pivot_table = pivot_table.head(n)
    fig = px.bar(pivot_table, x = pivot_table[index], y = pivot_table[values], color = color)
    title = 'Efficieny of Service Delivery by Garage'.format(index)
    fig.update_layout(title = title, width = 1000, height = 900)
    fig.show()

In [34]:
make_pivot_bar_top_garage(df, 'City', 'service_time_hours', np.mean, 20, 'service_time_hours')

In [35]:
#Define function to make pivot_table and bar graph for total amount spent by type
def make_pivot_bar_type(dataframe, index1, index2, values, aggfunc, color):
    global pivot_table
    pivot_table = pd.pivot_table(data = dataframe, index = [index1, index2], values = values,
                                 aggfunc = aggfunc).reset_index().sort_values(by = values, ascending = False)
    pivot_table = pivot_table[pivot_table[values] > 0]
    fig = px.bar(pivot_table, x = pivot_table[index1], y = pivot_table[values], color = color)
    title = 'Amount spent per {} for each {}.'.format(index2, index1)
    fig.update_layout(title = title, width = 1000, height = 900, barmode = 'stack', xaxis_type = 'category')
    fig.update_xaxes(tickangle=315)
    fig.show()

In [36]:
make_pivot_bar_type(df, 'km_bins', 'Order Type', 'Total', 'sum', 'Order Type')

In [37]:
make_pivot_bar_type(df, 'District', 'Order Type', 'Total', 'sum', 'Order Type')

In [56]:
make_pivot_bar_type(df, 'Make', 'Order Type', 'Total', 'sum', 'Order Type')

In [38]:
#View df
df.head()

Cust Type Customer No.     District Invoice Date  Invoice No Invoice Time  \
0    Retail        67849  Maharashtra   02-01-2015  7005200002     11:30:36   
1    Retail        84419  Maharashtra   03-01-2015  7005200003     10:07:32   
2    Retail        81055  Maharashtra   03-01-2015  7005200004     11:12:57   
3    Retail        84419  Maharashtra   03-01-2015  7005200005     11:40:44   
4    Retail        18980  Maharashtra   03-01-2015  7005200006     12:07:17   

   Job Card No JobCard Date JobCard Time  KMs Reading  Labour Total  \
0       168303   13-12-2014     14:29:43        49317       1203.14   
1       173997   02-01-2015     14:12:18        78584        804.26   
2       173889   02-01-2015     11:40:44        33985        180.19   
3       174158   03-01-2015     10:12:32        78584          0.00   
4       173860   02-01-2015     10:45:42        50057       1202.97   

             Make  Misc Total        Model  OSL Total         Order Type  \
0  GENERAL MOTORS        0.00        SPARK     500.06       Paid Service   
1     TATA MOTORS      197.03       INDICA       0.00  SMC Value Package   
2   MARUTI SUZUKI        0.00          ZEN       0.00    Running Repairs   
3     TATA MOTORS        0.00       INDICA       0.00     SMC Redemption   
4         HYUNDAI      100.00  SANTRO XING     499.99       Paid Service   

   Parts Total  Recovrbl Exp   City   invoice_date_time   jobcard_date_time  \
0      2348.75           0.0  thane 2015-02-01 11:30:36 2014-12-13 14:29:43   
1         0.00           0.0  thane 2015-03-01 10:07:32 2015-02-01 14:12:18   
2        52.95           0.0  thane 2015-03-01 11:12:57 2015-02-01 11:40:44   
3         0.00           0.0  thane 2015-03-01 11:40:44 2015-03-01 10:12:32   
4       944.16           0.0  thane 2015-03-01 12:07:17 2015-02-01 10:45:42   

      service_time  service_time_hours  service_time_days    Total  car_age  \
0 49 days 21:00:53            1197.015               50.0  4051.95        4   
1 27 days 19:55:14             667.921               28.0  1001.29        7   
2 27 days 23:32:13             671.537               28.0   233.14        3   
3  0 days 01:28:12               1.470                0.0     0.00        7   
4 28 days 01:21:35             673.360               28.0  2747.12        5   

          km_bins  
0   20000 - 50000  
1  50000 - 100000  
2   20000 - 50000  
3  50000 - 100000  
4  50000 - 100000

In [39]:
make_pivot_bar_type(df, 'District', 'Make', 'Total', 'sum', 'Make')

In [40]:
#Define function for division
df['labour_to_parts'] = round(df['Labour Total']/df['Parts Total'],3)
df['labour_to_parts'] = df['labour_to_parts'].replace(np.nan, 0)

In [41]:
#Make bins for labour to parts ratio
df['ratio_bins'] = pd.cut(x=df['labour_to_parts'], bins=[-np.inf, 1, np.inf],
                      labels = ['Unprofitable', 'Profitable'])

In [42]:
#Show profitability of different order types
groupby = df.groupby(by = ['Order Type', 'ratio_bins'])[['Order Type']].size().reset_index(name = 'count')
fig = px.bar(groupby, x = 'Order Type', y = 'count', color = 'ratio_bins')
title = 'Profitability of different order types.'
fig.update_layout(title = title, width = 1000, height = 900, barmode = 'stack')
fig.update_xaxes(tickangle=315)
fig.show()

In [54]:
pivot_table_labour = pd.pivot_table(data = df, index = 'District', values = 'Labour Total',
                             aggfunc = np.mean).reset_index().sort_values(by = 'Labour Total', ascending = False)
fig = px.bar(pivot_table_labour, x = pivot_table_labour['District'], y = pivot_table_labour['Labour Total'],
             color = 'Labour Total')
title = 'Total Amount spent on Labour by District'
fig.update_layout(title = title, width = 1000, height = 900)
fig.show()

In [55]:
pivot_table_labour = pd.pivot_table(data = df, index = 'District', values = 'Parts Total',
                             aggfunc = np.mean).reset_index().sort_values(by = 'Parts Total', ascending = False)
fig = px.bar(pivot_table_labour, x = pivot_table_labour['District'], y = pivot_table_labour['Parts Total'],
             color = 'Parts Total')
title = 'Total Amount spent on Parts by District'
fig.update_layout(title = title, width = 1000, height = 900)
fig.show()

In [43]:
#Bar graph for average amount amount spend on labour by district
make_pivot_bar(df, 'District', 'Labour Total', np.mean, 'Average', 'Labour Total')

In [44]:
#Bar graph for average amount amount spend on labour by district
make_pivot_bar(df, 'District', 'Parts Total', np.mean, 'Average', 'Parts Total')

In [45]:
#Extract month column from JobCardDate
df['repair month'] = pd.DatetimeIndex(df['JobCard Date']).month

In [48]:
#Define function to make pivot_table and bar graph for total amount spent by type
def make_pivot_bar_type2(dataframe, index1, index2, values, aggfunc, color):
    global pivot_table
    pivot_table = pd.pivot_table(data = dataframe, index = [index1, index2], values = values,
                                 aggfunc = aggfunc).reset_index()
    pivot_table = pivot_table[pivot_table[values] > 0]
    fig = px.bar(pivot_table, x = pivot_table[index1], y = pivot_table[values], color = color)
    title = 'Amount spent per {} for each {}.'.format(index2, index1)
    fig.update_layout(title = title, width = 1000, height = 900, barmode = 'stack', xaxis_type = 'category')
    fig.update_xaxes(tickangle=315)
    fig.show()

In [58]:
#Make bar chart of order types by month of the year
make_pivot_bar_type2(df, 'repair month', 'Order Type', 'Total', 'sum', 'Order Type')

In [47]:
#Make bar chart of order types by month of the year
make_pivot_bar_type(df, 'Make', 'District', 'Total', 'sum', 'District')

In [39]:
#Make column for neagtive costs
df['neg cost'] = -df['Parts Total']
df['pos cost'] = df['Labour Total'] + df['OSL Total'] + df['Misc Total']

In [47]:
#Plot a graph for income vs outgo
pivot_costs = pd.pivot_table(df, index = 'Order Type', values = ['pos cost', 'neg cost'], aggfunc = np.mean).reset_index()
pivot_costs = pivot_costs[:-1]
fig = go.Figure(data =[go.Bar(name = 'Average Income', x = pivot_costs['Order Type'], y = pivot_costs['pos cost']),
                       go.Bar(name = 'Average Outgo', x = pivot_costs['Order Type'], y = pivot_costs['neg cost'])])
fig.update_layout(title = 'Profitability of Order Types', width = 1000, height = 800)

fig.show()

In [63]:
make_district

City                                                    \
Make                  AUDI   BMW DAEWOO    FIAT FORCE    FORD GENERAL MOTORS   
District                                                                       
Andaman and Nico.In.   NaN   NaN    NaN     2.0   NaN     NaN            NaN   
Andhra Pradesh        12.0  11.0   49.0   611.0  24.0  2046.0         2405.0   
Arunachal Pradesh      NaN   NaN    NaN     NaN   NaN     NaN            5.0   
Assam                  NaN   NaN    NaN     1.0   NaN     5.0            3.0   
Bihar                  NaN   NaN    NaN     7.0   1.0    31.0          154.0   
Chandigarh             1.0   2.0    9.0    54.0   1.0   144.0          214.0   
Chhattisgarh           NaN   2.0    NaN     3.0   NaN    24.0           18.0   
Dadra and Nagar Hav.   NaN   NaN    NaN    28.0   NaN    25.0           78.0   
Daman and Diu          NaN   1.0    NaN    27.0   NaN     4.0           22.0   
Delhi                  2.0   4.0    2.0    52.0   NaN   121.0          321.0   
Goa                    NaN   NaN    NaN     NaN   NaN     2.0            NaN   
Gujarat                9.0  27.0   18.0   614.0   7.0   675.0         1365.0   
Haryana                2.0   2.0   16.0   156.0   6.0   445.0         1105.0   
Himachal Pradesh       4.0   3.0    3.0    49.0   4.0   136.0          285.0   
Jammu and Kashmir      NaN   NaN    NaN     NaN   NaN     NaN            1.0   
Jharkhand              NaN   2.0    NaN     4.0   NaN    10.0           14.0   
Karnataka             14.0  11.0   54.0   899.0  18.0  2764.0         2777.0   
Kerala                 1.0  11.0   20.0   104.0   8.0   329.0          338.0   
Lakshadweep            NaN   NaN    NaN     NaN   NaN     NaN            1.0   
Madhya Pradesh        22.0   8.0   11.0    81.0  14.0   447.0          576.0   
Maharashtra           60.0  66.0  103.0  1742.0  33.0  4602.0         8922.0   
Manipur                NaN   NaN    NaN     NaN   NaN     NaN            NaN   
Megalaya               NaN   NaN    NaN     NaN   NaN     NaN            NaN   
Mizoram                NaN   NaN    NaN     NaN   NaN     NaN            NaN   
Nagaland               NaN   NaN    NaN     NaN   NaN     NaN            NaN   
Odisha                 1.0   NaN    1.0     8.0   NaN    98.0          236.0   
Puducherry             NaN   NaN    NaN     9.0   NaN   126.0          194.0   
Punjab                42.0  54.0   11.0   255.0   5.0   832.0         1489.0   
Rajasthan              4.0   5.0    2.0    61.0  11.0   383.0          650.0   
Sikkim                 NaN   NaN    NaN     NaN   NaN     NaN            NaN   
Tamil Nadu            34.0  41.0   73.0  1190.0  64.0  7259.0         5497.0   
Telangana              NaN   2.0    3.0   178.0   2.0   287.0          553.0   
Tripura                NaN   NaN    NaN     NaN   NaN     NaN            NaN   
Uttar Pradesh         12.0   5.0   16.0   281.0   6.0   678.0         1734.0   
Uttarakhand            NaN   NaN    1.0    15.0   NaN   119.0           73.0   
West Bengal            NaN   NaN    NaN     6.0   3.0    24.0           62.0   

                                                                        \
Make                 HINDUSTAN MOTORS   HONDA  HYUNDAI JEEP LAND ROVER   
District                                                                 
Andaman and Nico.In.              NaN     NaN      NaN  NaN        NaN   
Andhra Pradesh                   15.0   856.0   5933.0  NaN        1.0   
Arunachal Pradesh                 NaN     NaN      5.0  NaN        NaN   
Assam                             NaN     2.0     22.0  NaN        NaN   
Bihar                             NaN    34.0    162.0  1.0        NaN   
Chandigarh                        NaN   121.0    737.0  NaN        NaN   
Chhattisgarh                      NaN    33.0    106.0  NaN        NaN   
Dadra and Nagar Hav.              NaN    16.0    154.0  NaN        NaN   
Daman and Diu                     NaN    20.0     89.0  NaN        1.0   
Delhi        